In [1]:
# Import necessary libraries: pandas for data handling, requests for API requests, folium for map visualization.
import pandas as pd
import requests
import folium
import sys

# Load the city data from the CSV file.
city_data_df = pd.read_csv('https://raw.githubusercontent.com/GBov81/python-api-challenge/main/output_data/cities.csv')

# Create a map centered on a location (e.g., the first city in the DataFrame).
m = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=5)

# Add city points to the map based on humidity and display the map.
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  # Adjust the size of the circle marker based on humidity.
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"City: {row['City']}<br>Humidity: {row['Humidity']}%"
    ).add_to(m)
m

In [2]:
# Filter the DataFrame to find cities with ideal weather conditions and display the DataFrame.
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 21) &
                                (city_data_df['Max Temp'] < 27) &
                                (city_data_df['Wind Speed'] < 4.5) &
                                (city_data_df['Cloudiness'] == 0)]
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
81,81,makakilo city,21.3469,-158.0858,21.66,81,0,2.57,US,1666108282
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356
211,211,laguna,38.4210,-121.4238,21.67,79,0,2.06,US,1666108364
240,240,tikaitnagar,26.9500,81.5833,23.56,59,0,0.35,IN,1666108378
265,265,san quintin,30.4833,-115.9500,21.20,74,0,1.37,MX,1666108394
340,340,santa rosalia,27.3167,-112.2833,24.62,56,0,0.74,MX,1666108436


In [3]:
# Create an empty list to store hotel information.
hotel_data = []

# Iterate through each city in ideal_weather_df and find nearby hotels using Geoapify Places API.
for index, row in ideal_weather_df.iterrows():
    # Construct the Geoapify Places API URL to find hotels within a 10,000-meter radius.
    url = f'https://api.geoapify.com/v2/places?lat={row["Lat"]}&lon={row["Lng"]}&categories=accommodation.hotel&limit=1&radius=10000&apiKey=06f23ac28d4b4265aaa0df88e4af3b54'
    
    # Send a GET request to the API and retrieve the response data.
    response = requests.get(url)
    data = response.json()
    
    # Check if hotel data is available in the API response.
    if 'features' in data and len(data['features']) > 0:
        hotel_name = data['features'][0]['properties']['name']  
        country = data['features'][0]['properties']['country']  
    
    # Append city and hotel information to the hotel_data list.
    hotel_data.append({'City': row['City'], 'Country': country, 'Latitude': row['Lat'], 'Longitude': row['Lng'],'Hotel Name': hotel_name})

# Convert the list of dictionaries into a DataFrame and display it.
hotel_df = pd.DataFrame(hotel_data)
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,kapaa,United States,22.0752,-159.3190,Pono Kai Resort
1,hilo,United States,19.7297,-155.0900,Dolphin Bay Hotel
2,banda,India,25.4833,80.3333,#acnindiafy21
3,makakilo city,United States,21.3469,-158.0858,Embassy Suites by Hilton Oahu Kapolei
4,kahului,United States,20.8947,-156.4700,Maui Seaside Hotel
5,gat,Israel,31.6100,34.7642,מלון גולדן טאוור
6,laguna,United States,38.4210,-121.4238,Holiday Inn Express & Suites
7,tikaitnagar,India,26.9500,81.5833,Maa patesawri sweet hous
8,san quintin,Mexico,30.4833,-115.9500,Jardines Hotel
9,santa rosalia,Mexico,27.3167,-112.2833,Sol y Mar
